In [1]:
from ddpm import DDPMSampler
import torch
import numpy as np
import math
import torchaudio
from IPython.display import Audio


generator = torch.Generator()
generator.manual_seed(0)

ddpm_sampler = DDPMSampler(generator)

# How many noise levels to generate
noise_levels = [0, 10, 50, 75, 100, 250, 500, 750]

# Load the audio file (replace 'your_audio_file.wav' with the actual file path)
waveform, sample_rate = torchaudio.load("../audio/rim.wav")
#waveform, sample_rate = librosa.load("../audio/rim.wav", sr=None)



# Normalize the waveform to the range [-1, 1]
waveform = waveform / torch.max(torch.abs(waveform))

# Create a batch by repeating the same waveform many times
batch = waveform.repeat(len(noise_levels), 1)

ts = torch.tensor(noise_levels, dtype=torch.int, device=batch.device)
noise_waveforms = []
epsilons = torch.randn(batch.shape, device=batch.device)

# Generate a noisified version of the waveform for each noise level
for i in range(len(ts)):
    a_hat = ddpm_sampler.alphas_cumprod[ts[i]]
    noise_waveforms.append(
        (torch.sqrt(a_hat) * batch[i]) + (torch.sqrt(1 - a_hat) * epsilons[i])
    )

noise_waveforms = torch.stack(noise_waveforms, dim=0)
noise_waveforms = torch.clamp(noise_waveforms, -1, 1)

# You may want to save the noisified waveforms or further process them depending on your needs
# For demonstration purposes, let's print the noisified waveform shape
print("Noisified Waveform Shape:", noise_waveforms.shape)
Audio(noise_waveforms[7], rate=sample_rate)


torch.Size([2, 4629])
Noisified Waveform Shape: torch.Size([8, 4629])
